# Camera Calibration Initialization
For 3D pose tracking, we need to know how the image from each camera relates to the scene playing out in the world. To understand this relationship, we need to estimate an array of camera parameters, which will then be optimized using the `pySBA.py` bundle adjustment algorithm. These are just initialization parameters, so they don't need to be perfect. Instead, we will aim for estimating them in a way basically reflects reality while keeping the math easy.


The first set of parameters are the camera **extrinsics**. These define where each camera is relative to the world (the arena). They can be broken down into a *rotation* (how the world coordinate axes relate to the camera coordinate axes) and a *translation* (how the world origin relates to the camera origin). We will estimate these empirically below.


The second set of parameters are the camera **intrinsics**. These define how the 3D scene is transformed into a 2D image on the camera sensor. These include the focal length (distance between the lens and the camera sensor), the x and y principle point offsets (how the image center relates to the center of the lens), and the axis skew (a measurement of shear distortion). These are all measured in pixels. We'll calculate these parameters below using what we know about the physical lens, camera sensor, and image cropping.


**Resources:**
- Selmaan's [evernote document](https://www.evernote.com/shard/s552/sh/ec4d276d-14de-43e4-b5c9-61f2404d9951/8d825d89cb5b9b1cc9099363bfa6d8da![image.png](attachment:image.png) on how he calibrated his cameras.
- 3-part [blog post](http://ksimek.github.io/2012/08/14/decompose/) about camera extrinsics and intrinsics.
- [Laser calibration](https://github.com/JohnsonLabJanelia/laserCalib) github repo from Rob Johnson + Jinyao (Jane) Yan at Janelia (largely copied/modified in my `bird_pose_tracking\camera_calibration`).
- [Python bundle adjustment](https://github.com/jahdiel/pySBA/blob/master/PySBA.py) github repo.

## Empirically estimate the camera extrinsincs

These are needed to initialize the camera array for laser calibration and will be subsequently optimized during the calibration process. The extrinsics roughly define the location of the camera in the world and what direction its pointing in, using a rotation matrix and translation vector.

For both world and camera coordinates, we will use a normalized coordinate system where half the sidelength of the arena = 1.

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation

### Define the world coordinates
- origin is at the arena center
- $z$ is up from the floor
- $x$ increases towards the red and blue feeders
- $y$ increases towards the yellow and red feeders

So the red feeder is at $(x = 1, y = 1, z = 0)$.

### Define the camera coordinates
- origin is the camera center
- $x$ increases to the right of the camera
- $y$ increases down from the camera (**note** that world $z$ becomes camera $y$)
- $z$ increases away from the camera

We can approximate the cameras as being a bit above each of the 4 corners for easy math. In other words, let $h = 1/4$ be the height of the cameras above the arena and let $d = \sqrt{2}$ be the distance of each camera from the origin.


Then the translation vector needed to describe the world origin in terms of camera coordinates is simply $t = [0, h, d]$.

In [28]:
h = 1/4
d = np.sqrt(2)
t = np.asarray([0, h, d])

### Choose points
To find the rotation matrix, we need to choose a bunch of points where the world and camera coords are easy to estimate. We'll use the 4 corners, plus a point camera height above the origin. In order of red, yellow, green, blue, above-center.

Let $W$ be the world coords of the points $(x, y, z)$:

In [29]:
world_pts = np.asarray([[1, 1, 0], [-1, 1, 0], [-1, -1, 0], [1, -1, 0], [0, 0, h]])
print('points in world coordinates:\n')
print(world_pts)

points in world coordinates:

[[ 1.    1.    0.  ]
 [-1.    1.    0.  ]
 [-1.   -1.    0.  ]
 [ 1.   -1.    0.  ]
 [ 0.    0.    0.25]]


It's also easy to determine the camera coordinates of each of our chosen points for each camera:
- The point in the camera's own corner will be at the origin in $x$ and $z$, and $h$ below in y.
- The 2 corners adjacent to the camera are $d$ away in $x$ and $z$, and $h$ below.
    - Using the right-hand rule, the corner to the right is $d$ in $x$, while the corner to the left is $-d$.
- The furthest corner is $2d$ away in $z$, $0$ in $x$, and $h$ below.
- The above-center point is $d$ away in $z$, $0$ in $x$ and $y$.

Let $C$ be the camera coords of the points $(x, y, z)$:

In [55]:
red_pts = np.asarray([[0, h, 0], [d, h, d], [0, h, 2*d], [-d, h, d], [0, 0, d]])
yellow_pts = np.asarray([[-d, h, d], [0, h, 0], [d, h, d], [0, h, 2*d], [0, 0, d]])
green_pts = np.asarray([[0, h, 2*d], [-d, h, d], [0, h, 0], [d, h, d], [0, 0, d]])
blue_pts = np.asarray([[d, h, d], [0, h, 2*d], [-d, h, d], [0, h, 0], [0, 0, d]])

cam_pts = (red_pts, yellow_pts, green_pts, blue_pts)

print('points in red camera coordinates:\n')
print(np.round(red_pts, 4))

points in red camera coordinates:

[[ 0.      0.25    0.    ]
 [ 1.4142  0.25    1.4142]
 [ 0.      0.25    2.8284]
 [-1.4142  0.25    1.4142]
 [ 0.      0.      1.4142]]


### Find the rotation matrix
We can now use least squares to solve the system of equations and find the rotation matrix $R$ needed to rotate the world points $W$ onto their postitions $C$ in the camera frame of reference (accounting for the translation $t$ of each camera):

$$WR = C - t$$

Then, we'll convert this into a rotation vector for each camera (which is the format needed for the calibration algorithm).

In [56]:
n_cams = len(cam_pts)
n_pts = world_pts.shape[0]

rotation_mats = []
rotation_vecs = np.zeros((n_cams, 3))
for i, c in enumerate(cam_pts):
    [rot, tmp, tmp, tmp] = np.linalg.lstsq(a=world_pts, b=(c-t))
    rotation_mats.append(rot)
    r = Rotation.from_matrix(rot)
    rotation_vecs[i] = r.as_rotvec()
    
# for some reason, scipy's rotation formula gives a rotation vector that is sign-flipped...
rotation_vecs = -rotation_vecs

C:\Users\ilow1\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  import sys


In [57]:
''' Display the translation vector and the rotation vector for each camera '''
print(f'the translation vector for all cams is: {np.round(t, 3)}\n')

cams = ['red', 'yellow', 'green', 'blue']
for c, r in zip(cams, rotation_vecs):
    print(f'{c} cam rotation vector is: {np.round(r, 3)}')

the translation vector for all cams is: [0.    0.25  1.414]

red cam rotation vector is: [ 0.729  1.76  -1.76 ]
yellow cam rotation vector is: [ 0.729 -1.76   1.76 ]
green cam rotation vector is: [ 1.482 -0.614  0.614]
blue cam rotation vector is: [ 1.482  0.614 -0.614]


### Check our work
We can double check that the rotation vectors and matrices are correct by using them to rotate the points from world coordinates to each camera's coordinate frame. To rotate with the rotation vector, we need to use [Rodrigues' rotation formula](https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula). If all goes well, the three outputs below should be identical.

In [58]:
cam_idx = 1

In [59]:
def rodrigues_rot(v, k_rot):
    '''
    Rotate a given vector (v) by a rotation vector (k_rot) using Rodrigues' rotation formula.
    '''
    theta = np.linalg.norm(k_rot)
    k = k_rot / theta
    dot = v.dot(k)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    return cos_theta * v + sin_theta * np.cross(k, v) + dot * (1 - cos_theta) * k

In [60]:
''' check that the rotation vector works '''
rot_vec = rotation_vecs[cam_idx]
rot_vec_pts = np.zeros_like(red_pts)
for i, pts in enumerate(world_pts):
    rot_vec_pts[i] = rodrigues_rot(pts, rot_vec)
rot_vec_pts = rot_vec_pts + t

print(np.round(rot_vec_pts, 3))

[[-1.414  0.25   1.414]
 [-0.     0.25   0.   ]
 [ 1.414  0.25   1.414]
 [ 0.     0.25   2.828]
 [-0.     0.     1.414]]


In [61]:
''' check that the rotation matrix works '''
rot_mat_pts = world_pts.dot(rotation_mats[cam_idx]) + t
print(np.round(rot_mat_pts, 3))

[[-1.414  0.25   1.414]
 [-0.     0.25   0.   ]
 [ 1.414  0.25   1.414]
 [ 0.     0.25   2.828]
 [ 0.     0.     1.414]]


In [62]:
print(np.round(cam_pts[cam_idx], 3))

[[-1.414  0.25   1.414]
 [ 0.     0.25   0.   ]
 [ 1.414  0.25   1.414]
 [ 0.     0.25   2.828]
 [ 0.     0.     1.414]]


## Calculate the camera intrinsics
These define how the 3D scene is transformed into a 2D image on the camera sensor. They are all measured in pixels and are based on an approximation of the camera as a pinhole camera. They can be roughly calculated using the camera and lens specs.

They are:
- Focal distance $f_x$, $f_y$: distance between the "pinhole" and the image. These are assumed to be equal.
- Principle point offset $x_0$, $y_0$: 2D distance between the image center and the pinhole center.
- Axis skew $s$: a measure of distortion, which we will estimate as 0.


One useful framing of these parameters is in terms of a matrix decomposition that breaks down the contribution of each set of parameters to the final image. Consider a matrix of intrinsics $K$.

$$
K = \begin{bmatrix}f_x & s & x_0 \\ 0 & f_y & y_0\\ 0 & 0 & 1 \end{bmatrix}
$$


This matrix decomposes into a *2D translation* defined by $x_0$ and $y_0$, a *2D scaling* defined by $f_x$ and $f_y$, and a *2D shear* defined by $s$:

$$
K = \begin{bmatrix}1 & 0 & x_0 \\ 0 & 1 & y_0\\ 0 & 0 & 1 \end{bmatrix} * 
\begin{bmatrix}f_x & 0 & 0 \\ 0 & f_y & 0\\ 0 & 0 & 1 \end{bmatrix} * 
\begin{bmatrix}1 & \frac{s}{f_x} & 0 \\ 0 & 1 & 0\\ 0 & 0 & 1 \end{bmatrix}
$$


To calculate these things, you need to know what type of camera and lens you're using. I'm using a Flir [Blackfly BFS-U3-88S6C-C](https://www.edmundoptics.com/p/bfs-u3-88s6c-c-usb3-blackflyreg-s-color-camera/40168/), which has a [Sony IMX267](https://www.1stvision.com/cameras/sensor_specs/IMX267-sensor-spec.pdf) sensor, and a 12.5mm lens.

### Focal distance
This is the distance between the pinhole and the image, which in our case is basically the focal length of the lens. This should be the same for all the cameras unless you're using a different lens for any of them. To get the focal distance in pixels, we can use the focal length of the lens and the sensor dimensions. Let $W$ be the sensor width in mm, $w$ be the width in pixels, and $F_x$ be the focal length in mm. We can calculate focal distance in pixels, $f_x$ with:

$$f_x = F_x \frac{w}{W}$$

In [12]:
''' Define the lens and sensor params '''
# focal length of your lens
Fx = 12.5 # mm

# these are from the sensor datasheet and cropping params in Spinnaker
pixel_size = 6.9e-3 # mm - binned pixel size
w_pixels = 1896 # width of the cropped image in pixels

In [13]:
''' Calculate focal distance in pixels '''
# use pixel size and image width in pixels to determine W
W_mm = pixel_size * w_pixels # width of the cropped image in mm

# compute focal distance (pixels)
fx = Fx * (w_pixels / W_mm)

print(f'focal distance is {np.round(fx, 2)} pixels')

focal distance is 1811.59 pixels


### Principle point offset
This is just the pixel distance from the origin (top left corner in Blackfly cameras) to the image center. To compute this, simply divide your image height/width by 2 and add the x and y offset (all found in Spinnaker, image format). Note that this may be slightly different for each camera, depending on how they were positioned and cropped.

In [14]:
''' Input your image params '''
image_w = 1896 # pixels
image_h = 640 # pixels
x_shift = 80
y_shift = 40

# my blue camera is positioned a little differently from the others..
x_shift_blue = 48
y_shift_blue = 26

In [15]:
''' Calculate offsets '''
x0 = (image_w / 2) + x_shift
x0_blue = (image_w / 2) + x_shift_blue
y0 = (image_h / 2) + y_shift
y0_blue = (image_h / 2) + y_shift_blue

## Save everything in a camera array
Finally, we can stick all these params into an array for each camera, which we'll use to initialize the camera array in [pySBA](https://github.com/jahdiel/pySBA/blob/master/PySBA.py).


Camera parameters are:


**Extrinsics**
- A 3D rotation vector that rotates the world coordinate axes into camera coordinate axes; array of floats, shape (3, )
- A 3D translation vector that translates the world origin to the camera origin; array of floats, shape (3, )


**Intrinsics**
- Focal distance in pixels; float
- Distortion params; array of floats, shape (2, ) (we'll call these 0 for now)
- Principal point offsets (x, y); array of ints, shape (2, )

In [25]:
cam_ids = ['red_cam', 'yellow_cam', 'green_cam', 'blue_cam']
file_path = 'C:/Users/ilow1/Documents/Python Scripts/bird_pose_tracking/calibration_files/'
folder = 'init_cam_arrays/'

intrinsics = np.asarray([fx, 0, 0, x0, y0])
intrinsics_blue = np.asarray([fx, 0, 0, x0_blue, y0_blue])

for i, cam in enumerate(cam_ids):
    if cam == 'blue_cam':
        cam_array = np.append(rotation_vecs[i], np.append(t, intrinsics_blue))
    else:
        cam_array = np.append(rotation_vecs[i], np.append(t, intrinsics))
    np.save(f'{file_path}{folder}{cam}_array.npy', cam_array)